# Create and/or add data to tvde_earnings_history table in the bronze layer

In [1]:
# Importing libraries
import os
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import re
from datetime import datetime
from pyspark.sql import SparkSession
#from pyspark.sql.functions import col, lit, regexp_extract, when, to_date, unix_timestamp, from_unixtime, concat_ws, lpad, create_map
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType



In [2]:
# layer and table
layer = "bronze"
table = "tvde_earnings_history"
catalog_name = "toll_reconciliation_tool"

# Define the desired warehouse location explicitly
project_dir = "C:/Users/renat/Documents/imgPdados-finance-uber/toll-reconciliation-tool/spark-warehouse"
warehouse_location = "file:///" + os.path.abspath(project_dir)
print(f"Setting Spark warehouse to: {warehouse_location}")

# Change the current working directory
os.chdir(project_dir)
print(f"Changed current working directory to: {os.getcwd()}")

Setting Spark warehouse to: file:///C:\Users\renat\Documents\imgPdados-finance-uber\toll-reconciliation-tool\spark-warehouse
Changed current working directory to: C:\Users\renat\Documents\imgPdados-finance-uber\toll-reconciliation-tool\spark-warehouse


In [3]:
# --- Functions (select_files, add_files_to_list, normalize_dataframe) ---
def select_files():
    """Opens a file selection dialog and returns a list with the paths of the selected files."""
    root = tk.Tk()  # Create a Tkinter window
    root.withdraw()  # Hide the main window

    file_paths = filedialog.askopenfilenames(
        title="Select files",  # Window title
    )

    return list(file_paths)  # Convert the returned tuple to a list

def add_files_to_list(file_list):
    """Adds the paths of the selected files to the list."""
    selected_file_paths = select_files()

    if selected_file_paths:  # Check if the user selected any files
        file_list.extend(selected_file_paths)  # Add the paths to the list
        print("Files added:")
        for file_path in selected_file_paths:
            print(file_path)
    else:
        print("No files selected.")

def normalize_dataframe(df, filename):
    """Normalizes a DataFrame according to specified rules."""

    df['Date'] = None
    df['Earnings'] = None
    df['Toll'] = None
    df['ServiceFee'] = None
    df['Tip'] = None
    df['StartTime'] = None
    df['TotalTime'] = None
    df['Distance'] = None
    df['DataInput'] = None

    toll_words = ["pedágio"]
    service_fee_words = ["taxa de serviço"]
    tip_words = ["valor extra"]

    # Extract year and month from filename
    year_match = re.search(r'\d{4}', filename)
    month_match = re.search(r'(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)', filename, re.IGNORECASE)

    if year_match and month_match:
        year = year_match.group(0)
        month = month_match.group(0).lower()
        month_number = {
            'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 'mai': '05', 'jun': '06',
            'jul': '07', 'ago': '08', 'set': '09', 'out': '10', 'nov': '11', 'dez': '12'
        }[month]
    else:
        year = None
        month_number = None

    for index, row in df.iterrows():
        text = row['Extracted Text']

        # include control columns
        df.at[index, 'DataInput'] = datetime.now()

        # Date
        date_match = re.search(r'(seg|ter|qua|qui|sex|sáb|dom|Mon|Tue|Wed|Thu|Fri|Sat|Sun)?,? \d{2} de [a-z]{3}', text, re.IGNORECASE)
        if date_match:
            day_month = re.search(r'\d{2} de [a-z]{3}', date_match.group(0), re.IGNORECASE).group(0)
            day = re.search(r'\d{2}', day_month).group(0)
            if year and month_number:
                df.at[index, 'Date'] = f"{year}/{month_number}/{day}"
            else:
                df.at[index, 'Date'] = f"{day}" 
        else:
            df.at[index, 'Date'] = f'{year}/{month_number}/{day}'


        # Earnings
        earnings_match = re.search(r'€\s*([\d,.]+)', text)
        if earnings_match:
            df.at[index, 'Earnings'] = earnings_match.group(1).replace(',', '.')

        # Toll
        for word in toll_words:
            toll_match = re.search(r'([\d,.]+)\s*' + word, text, re.IGNORECASE)
            if toll_match:
                df.at[index, 'Toll'] = toll_match.group(1).replace(',', '.')
                break

        # Service Fee
        for word in service_fee_words:
            service_match = re.search(r'€\s*([\d,.]+)\s*' + word, text, re.IGNORECASE)
            if service_match:
                df.at[index, 'ServiceFee'] = service_match.group(1).replace(',', '.')
                break

        # Tip
        for word in tip_words:
            tip_match = re.search(r'([\d,.]+)\s*' + word, text, re.IGNORECASE)
            if tip_match:
                df.at[index, 'Tip'] = tip_match.group(1).replace(',', '.')
                break

        # Start Time
        start_time_match = re.search(r'(\d{2}-\d{2}|\d{2}\.\d{2}|\d{2}\*\d{2})', text)
        if start_time_match:
            df.at[index, 'StartTime'] = start_time_match.group(1).replace('.', ':').replace('*', ':').replace('-', ':')

        # Total Time
        total_time_match = re.search(r'(\d+)\s*min\s*(\d+)\s*seg', text)
        if total_time_match:
            df.at[index, 'TotalTime'] = f"{total_time_match.group(1)}:{total_time_match.group(2)}"

        # Distance
        distance_match = re.search(r'([\d,.]+)\s*km', text)
        if distance_match:
            df.at[index, 'Distance'] = distance_match.group(1).replace(',', '.')

    return df


In [4]:
# Selecting Files:
filepath_list = [] # Create an empty list
add_files_to_list(filepath_list) # Call the function to add files and print the final list
pandas_dataframes_list = []  # List to store DataFrames


Files added:
C:/Users/renat/Documents/imgPdados-finance-uber/toll-reconciliation-tool/2023-dezembro.csv


In [5]:
# puts each df in the 'dataframes_list', one for each CSV file.
for filepath in filepath_list:
    try:
        df = pd.read_csv(filepath, encoding='utf-8')  # Use utf-8 encoding
        pandas_dataframes_list.append(df)
        print(f"Successfully read {filepath}")
    except FileNotFoundError:
        print(f"Error: File not found - {filepath}")
    except pd.errors.EmptyDataError:
        print(f"Error: Empty CSV file - {filepath}")
    except pd.errors.ParserError:
        print(f"Error: Parsing error in {filepath}. Check the file format.")
    except Exception as e:
        print(f"An unexpected error occurred while reading {filepath}: {e}")



# You can access each DataFrame like this:
if pandas_dataframes_list:  # Check if the list is not empty
    for i,df in enumerate(pandas_dataframes_list):
        print(f"Number of rows in {df.shape}")
        print(f"\nFirst few rows of the {i+1}th DataFrame:")
        display(df.head())  # Print the first few rows of the first DataFrame
else:
    print("\nNo CSV files were successfully read.")


Successfully read C:/Users/renat/Documents/imgPdados-finance-uber/toll-reconciliation-tool/2023-dezembro.csv
Number of rows in (252, 1)

First few rows of the 1th DataFrame:


,Extracted Text
0,"sex , 29 de dez"
1,"€ 2,60 1l-46 Taxa de serviço"
2,"€3,23 21-23 UberX 6 min 22 segundos 138 km 4 €..."
3,"€5,22 UberX Saver . 18 min 46 segundos 20-12 8..."
4,"€4,37 UberX Saver ' 18 min 1l segundos 19.45 6..."


In [6]:
# normalizing Pandas DataFrame 
normalized_pandas_dataframes_list = [normalize_dataframe(df.copy(), os.path.basename(filepath)) for df, filepath in zip(pandas_dataframes_list, filepath_list)]


# Now 'normalized_dataframes' contains the normalized DataFrames.
# Configure pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Increase terminal width (adjust as needed)

# To view the first normalized DataFrame rows:
if normalized_pandas_dataframes_list:  # Check if the list is not empty
    for i,ndf in enumerate(normalized_pandas_dataframes_list):
        print(f"Number of rows in {ndf.shape}")
        print(f"\nFirst few rows of the {i+1}th DataFrame:")
        display(ndf.head())  # Print the first few rows of each DataFrame
else:
    print("\nNo files were successfully read.")


Number of rows in (252, 10)

First few rows of the 1th DataFrame:


,Extracted Text,Date,Earnings,Toll,ServiceFee,Tip,StartTime,TotalTime,Distance,DataInput
0,"sex , 29 de dez",2023/12/29,None,None,None,None,None,None,None,2025-04-20 22:34:23.826855
1,"€ 2,60 1l-46 Taxa de serviço",2023/12/29,2.60,None,None,None,None,None,None,2025-04-20 22:34:23.830844
2,"€3,23 21-23 UberX 6 min 22 segundos 138 km 4 €...",2023/12/29,3.23,None,None,0.50,21:23,6:22,138,2025-04-20 22:34:23.831843
3,"€5,22 UberX Saver . 18 min 46 segundos 20-12 8...",2023/12/29,5.22,None,None,None,20:12,18:46,8.22,2025-04-20 22:34:23.832840
4,"€4,37 UberX Saver ' 18 min 1l segundos 19.45 6...",2023/12/29,4.37,None,None,None,19:45,None,6.89,2025-04-20 22:34:23.833839


In [7]:
# Convertendo os tipos de dados no Pandas explicitamente
for ndf in normalized_pandas_dataframes_list:
    numeric_cols = ['Earnings', 'Toll', 'ServiceFee', 'Tip', 'Distance']
    for col in numeric_cols:
        ndf[col] = pd.to_numeric(ndf[col], errors='coerce')
    ndf['DataInput'] = pd.to_datetime(ndf['DataInput'])
    string_cols = ['Extracted Text', 'Date', 'StartTime', 'TotalTime']
    for col in string_cols:
        ndf[col] = ndf[col].astype(str)
    print(ndf.dtypes) # Verifique os tipos após a conversão

Extracted Text            object
Date                      object
Earnings                 float64
Toll                     float64
ServiceFee               float64
Tip                      float64
StartTime                 object
TotalTime                 object
Distance                 float64
DataInput         datetime64[ns]
dtype: object


In [8]:
# Initialize Spark session with the specified warehouse directory
spark = SparkSession.builder \
    .appName(catalog_name) \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()


# Confirming that the session is working
if spark:
    print("Spark session started successfully!")
    print(f"Application name: {spark.sparkContext.appName}")
else:
    print("Failed to start Spark session.")


Spark session started successfully!
Application name: toll_reconciliation_tool


In [9]:
# Attempt a basic Hive operation
try:
    # Show a list of layers/schemas/databases in the spark-warehouse
    spark.sql("SHOW DATABASES").show()
except Exception as e:
    print(f"Error: {e}")

# Check if the 'bronze' schema exists
result = spark.sql("SHOW SCHEMAS").collect()
schemas = [row[0] for row in result]

if "bronze" not in schemas:
    print("Error: The 'bronze' schema does not exist. Please create it.")  # error for not finding the bronze layer
else:
    print("The 'bronze' schema exists.")
    # Switch to the bronze layer
    spark.sql(f"USE {layer}")



+---------+
|namespace|
+---------+
|   bronze|
|  default|
|     gold|
|   silver|
+---------+

The 'bronze' schema exists.


In [10]:

# Convert each Pandas DataFrame in the list to a Spark DataFrame
spark_dataframes_list = [spark.createDataFrame(ndf) for ndf in normalized_pandas_dataframes_list]


In [11]:

# To view the first normalized Spark DataFrame rows:
if spark_dataframes_list:  # Check if the list is not empty
    for i, sdf in enumerate(spark_dataframes_list):
        print(f"Visualizando DataFrame número: {i + 1} da lista")
        print("\nPrimeiras 5 linhas:")
        sdf.show(5)
        print("-" * 30)  # Separador para melhor visualizaçã
else:
    print("\nNo files were successfully read.")

Visualizando DataFrame número: 1 da lista

Primeiras 5 linhas:


Py4JJavaError: An error occurred while calling o62.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (Renato-Laptop-LenovoIdeapad320-SSD executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.com

In [ ]:

for sdf in spark_dataframes_list:
    try:
        # Write the normalized Spark DataFrame to a Delta table
        sdf.write \
            .format('delta') \
            .mode('append') \
            .option('mergeSchema', 'true') \
            .saveAsTable(f'{layer}.{table}')
        print(f"Successfully processed and loaded {filepath} into {layer}.{table}")
            
    except Exception as e:
        print(f"An error occurred while processing {filepath}: {e}")
        import traceback
        traceback.print_exc()

print(f"All {len(filepath_list)} files processed (attempted) and loaded into '{layer}.{table}'.")


In [0]:
%sql
DESC DETAIL bronze.tvde_earnings_history

In [0]:
%sql
select *
from bronze.tvde_earnings_history

In [ ]:
# Stop the Spark session
spark.stop()